# Sentiment model with CNNs
## CAUTION: Only python 2.7!!

In [1]:
#Imports 
from __future__ import print_function

import tensorflow as tf
print(tf.__version__)

1.0.0


## Read data and create sequences

In [2]:
#Imports 
import numpy as np
import os
import shutil
import fnmatch

from collections import OrderedDict
from nltk import word_tokenize

data_path='/home/ubuntu/data/training/keras/aclImdb/'



# Generator of list of files in a folder and subfolders
def gen_find(filepath,top):
    for path, dirlist, filelist in os.walk(top):
        for name in fnmatch.filter(filelist,filepath):
            yield os.path.join(path,name)

def read_sentences(path):
    sentences = []
    sentences_list = gen_find("*.txt", path)
    for ff in sentences_list:
        with open(ff, 'r') as f:
            sentences.append(f.readline().strip())
    return sentences 

def tokenize(sentences):
    print( 'Tokenizing...',)
    tokens = []
    for sentence in sentences:
        tokens += [word_tokenize(sentence.decode('utf-8'))]
    print('Done!')
    return tokens

def build_dict(sentences):
    print( 'Building dictionary..',)
    wordcount = dict()
    for ss in sentences:
        for w in ss:
            if w not in wordcount:
                wordcount[w] = 1
            else:
                wordcount[w] += 1

    counts = wordcount.values()
    keys = wordcount.keys()
    sorted_idx = np.argsort(counts)[::-1]

    worddict = dict()
    for idx, ss in enumerate(sorted_idx):
        worddict[keys[ss]] = idx+1  # leave 0 (UNK)
    print( np.sum(counts), ' total words ', len(keys), ' unique words')
    return worddict, wordcount

def generate_sequence(sentences, dictionary):
    seqs = [None] * len(sentences)
    for idx, ss in enumerate(sentences):
        seqs[idx] = [dictionary[w] if w in dictionary else 1 for w in ss]
    return seqs

#Data extraction

#Extract training sentences
sentences_trn_pos = tokenize(read_sentences(data_path+'train/pos/'))
sentences_trn_neg = tokenize(read_sentences(data_path+'train/neg/'))
sentences_trn = sentences_trn_pos + sentences_trn_neg

#Build train dictionary
worddict, wordcount = build_dict(sentences_trn)

#Generate train data
train_x_pos = generate_sequence(sentences_trn_pos, worddict)
train_x_neg = generate_sequence(sentences_trn_neg, worddict)
X_train_full = train_x_pos + train_x_neg
y_trn = np.array([[1.,0.]]*len(train_x_pos) + [[0.,1.]]*len(train_x_neg), dtype=np.float32)


#Read test sentences and generate target y
sentences_tst_pos = read_sentences(data_path+'test/pos/')
sentences_tst_neg = read_sentences(data_path+'test/neg/')

test_x_pos = generate_sequence(tokenize(sentences_tst_pos), worddict)
test_x_neg = generate_sequence(tokenize(sentences_tst_neg), worddict)
X_test_full = test_x_pos + test_x_neg
y_tst = np.array([[1.,0.]]*len(test_x_pos) + [[0.,1.]]*len(test_x_neg), dtype=np.float32)


print('Preprocess done!')

Tokenizing...
Done!
Tokenizing...
Done!
Building dictionary..
7056193  total words  135098  unique words
Tokenizing...
Done!
Tokenizing...
Done!
Preprocess done!


In [3]:
max_features = 20000 # Number of most frequent words selected. the less frequent recode to 0
max_len = 100  # cut texts after this number of words (among top max_features most common words)


#Select the most frequent max_features, recode others using 0
def remove_features(x):
    return [[0 if w >= max_features else w for w in sen] for sen in x]

X_train = remove_features(X_train_full)
X_test  = remove_features(X_test_full)


# Shuffle data
from sklearn.utils import shuffle
X_train, y_trn = shuffle(X_train, y_trn, random_state=0)


# Cut or complete the sentences to length = maxlen
from keras.preprocessing import sequence

print("Pad sequences (samples x time)")
X_trn = sequence.pad_sequences(X_train, maxlen=max_len)
X_tst = sequence.pad_sequences(X_test, maxlen=max_len)
print('X_train shape:', X_trn.shape)
print('X_test shape:', X_tst.shape)


Using TensorFlow backend.


Pad sequences (samples x time)
X_train shape: (25000, 100)
X_test shape: (25000, 100)


## The model

In [4]:
# Model parameters

sequence_length = 100
vocab_size = 5000

embedding_size = 128
num_filters = 128
filter_sizes = [3, 4, 5]

LOG_DIR = '/tmp/tensorboard/sentiment_cnn_2'



In [5]:
# Start an interactive session
gpu_options = tf.GPUOptions(allow_growth = True)
sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=True))

In [6]:
# Inputs
input_x = tf.placeholder(tf.int32, shape=[None, sequence_length], name="input_x")
input_y = tf.placeholder(tf.int32, shape=[None, 2], name="input_y")
dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")

# Embedding layer
with tf.name_scope("embedding"):
    W_embedding = tf.Variable(tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0), name="W_embedding")
    embedded_chars = tf.nn.embedding_lookup(W_embedding, input_x)
    embedded_chars_expanded = tf.expand_dims(embedded_chars, -1)
    
   
# Create a convolution + maxpool layer for each filter size
def conv_layer(x, size_x=2, size_y=2, input_channels=1, output_channels=32):
    W_conv = tf.Variable(tf.truncated_normal([size_x, size_y, input_channels, output_channels], stddev=0.1), name='W')
    b_conv = tf.Variable(tf.constant(0.1, shape=[output_channels]), name='b')
    conv_out = tf.nn.relu(tf.nn.conv2d(x, W_conv, strides=[1, 1, 1, 1], padding='VALID') + b_conv, name='conv')
    pooled = tf.nn.max_pool(conv_out, ksize=[1, sequence_length - filter_size + 1, 1, 1], 
                            strides=[1, 1, 1, 1], padding='VALID', name="pool")
    return pooled

pooled_outputs = []
for i, filter_size in enumerate(filter_sizes):
    with tf.name_scope("conv-maxpool-%s" % filter_size):
        pooled = conv_layer(embedded_chars_expanded, size_x=filter_size, size_y=embedding_size, input_channels=1, output_channels=num_filters)
        pooled_outputs.append(pooled)

# Combine all the pooled features
num_filters_total = num_filters * len(filter_sizes)
h_pool = tf.concat(pooled_outputs, 3)
h_pool_flat = tf.reshape(h_pool, [-1, num_filters_total])    

# Add dropout
with tf.name_scope("dropout"):
    h_drop = tf.nn.dropout(h_pool_flat, dropout_keep_prob)

# Final (unnormalized) scores and predictions
with tf.name_scope("output"):
    W = tf.get_variable("W", shape=[num_filters_total, 2], initializer=tf.contrib.layers.xavier_initializer())
    b = tf.Variable(tf.constant(0.1, shape=[2]), name="b")
    
    l2_loss = tf.nn.l2_loss(W)
    l2_loss += tf.nn.l2_loss(b)

    scores = tf.nn.xw_plus_b(h_drop, W, b, name="scores")
    predictions = tf.argmax(scores, 1, name="predictions")
    
# Calculate Mean cross-entropy loss
loss_factor = 0.1

with tf.name_scope("loss"):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=scores, labels=input_y), name='loss') + loss_factor * l2_loss
    loss_summ = tf.summary.scalar("Loss", loss) #TENSORBOARD

# Accuracy
with tf.name_scope("accuracy"):
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predictions, tf.argmax(input_y, 1)), "float"), name="accuracy")
    acc_summ = tf.summary.scalar("Accuracy", accuracy) #TENSORBOARD
    
#Optimizer
with tf.name_scope("train") as scope:
    train_step = tf.train.AdamOptimizer(1e-3).minimize(loss)



In [7]:
# Merge all the summaries and write them out to /tmp/mnist_logs
with tf.name_scope('summaries') as scope:
    merged = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter(LOG_DIR + '/train', sess.graph)
    test_writer  = tf.summary.FileWriter(LOG_DIR + '/test')


In [8]:
# Define the embeddings in tensorboard
from tensorflow.contrib.tensorboard.plugins import projector


config = projector.ProjectorConfig()

# Create the embedding tensorboard container
embedding = config.embeddings.add()

# Link to the embeddings tensor
embedding.tensor_name = W_embedding.name

# Link this tensor to its metadata file (e.g. labels).
embedding.metadata_path = LOG_DIR + '/train/records.tsv'

# Saves a configuration file that TensorBoard will read during startup.
projector.visualize_embeddings(train_writer, config)


In [9]:
# Save an embeddings metadata file to visualize the words

# Inverse dictionary
idx2w = {v: k for k, v in worddict.iteritems()}

with open(LOG_DIR + '/train/records.tsv', "w") as record_file:
    record_file.write("UNK\n")
    for item in idx2w.items()[:4999]:
        record_file.write(item[1].encode('utf-8')+'\n')
        

## Batch iterator

In [10]:
def batch_iter(X, y, batch_size):
    """
    Generates a batch iterator for inputs and targets.
    """
    data_size = len(X)
    # Shuffle the data at each epoch
    shuffle_indices = np.random.permutation(np.arange(data_size))    
    shuffled_X = X[shuffle_indices]
    shuffled_y = y[shuffle_indices]
        
    num_batches = int((data_size-1)/batch_size) + 1
    for batch_num in range(num_batches):
        start_index = batch_num * batch_size
        end_index = min((batch_num + 1) * batch_size, data_size)
        yield shuffled_X[start_index:end_index], shuffled_y[start_index:end_index]


## Train iterator

In [12]:
#Inicialization.
sess.run(tf.global_variables_initializer())

# Train proccess
num_epochs = 20
batch_size = 128

step_trn = 0
step_tst = 0
print('e- LssTrn - AccTrn - LssTst - AccTst' )
for epoch in range(num_epochs):
    loss_trn = []
    acc_trn = []
    loss_tst = []
    acc_tst = []
    for x_batch, y_batch in batch_iter(X_trn, y_trn, batch_size):
        train_step.run(feed_dict={input_x: x_batch, input_y: y_batch, dropout_keep_prob: 0.5})
        loss_step, acc_step = sess.run([loss, accuracy], 
                                       feed_dict={input_x: x_batch, input_y: y_batch, dropout_keep_prob: 1})
        loss_trn += [loss_step]
        acc_trn += [acc_step]
        
        # Summary over the last batch of the epoch
        step_trn += 1
        if step_trn % 10 ==0 :
            summary_str = merged.eval(feed_dict={input_x: x_batch, input_y: y_batch, dropout_keep_prob: 1})
            train_writer.add_summary(summary_str, step_trn) #TENSORBOARD
        
    
    for x_batch_test, y_batch_test in batch_iter(X_tst, y_tst, batch_size):
        loss_step, acc_step = sess.run([loss, accuracy], 
                                       feed_dict={input_x: x_batch_test, input_y: y_batch_test, dropout_keep_prob: 1})
        loss_tst += [loss_step]
        acc_tst += [acc_step]
        
        # Summary over the last batch of the epoch
        step_tst += 1 
        if step_tst % 10 ==0 :
            summary_str = merged.eval(feed_dict={input_x: x_batch_test, input_y: y_batch_test, dropout_keep_prob: 1})
            test_writer.add_summary(summary_str, step_tst) #TENSORBOARD

    print(epoch, np.mean(loss_trn), np.mean(acc_trn), np.mean(loss_tst), np.mean(acc_tst))
    

e- LssTrn - AccTrn - LssTst - AccTst
0 0.935745 0.640649 0.67337 0.713911
1 0.587539 0.773079 0.613319 0.730971
2 0.558796 0.792044 0.61394 0.71808
3 0.52817 0.79886 0.550828 0.759957
4 0.481086 0.823398 0.506048 0.792953
5 0.435637 0.840466 0.495268 0.7875
6 0.391743 0.858801 0.443148 0.818487
7 0.347486 0.880891 0.431723 0.819842
8 0.310853 0.894228 0.41487 0.823453
9 0.272885 0.90731 0.402776 0.830708
10 0.239042 0.922027 0.393122 0.834885
11 0.209894 0.937779 0.380723 0.838728
12 0.182606 0.946939 0.384716 0.840426
13 0.156563 0.959566 0.389988 0.840824
14 0.136969 0.969858 0.398406 0.840689
15 0.118015 0.977559 0.411199 0.839206
16 0.101639 0.985611 0.428664 0.839246
17 0.0896087 0.990107 0.439062 0.837954
18 0.0804372 0.993543 0.450826 0.836288
19 0.0728485 0.996014 0.46527 0.837245


In [13]:
# Save checkpoint to visualize embeddings
saver = tf.train.Saver()
saver.save(sess, LOG_DIR + '/train/model.ckpt', 0)


'/tmp/tensorboard/sentiment_cnn_2/train/model.ckpt-0'